<a href="https://colab.research.google.com/github/florivz/DDM-Project-WS24-25/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/florivz/DDM-Project-WS24-25.git

Cloning into 'DDM-Project-WS24-25'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 29 (delta 9), reused 13 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 28.58 MiB | 14.60 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Updating files: 100% (10/10), done.


In [2]:
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from google.colab import userdata
from sklearn.cluster import KMeans
from matplotlib.patches import Patch
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import TruncatedSVD

# Loading the Data

In [3]:
pkl_path = 'DDM-Project-WS24-25/pkl/'

In [4]:
gdp_data = pd.read_pickle(pkl_path + "gdp.pickle")
df_pattern = pd.read_pickle(pkl_path + "naics_pattern.pickle")
df_occupation_1 = pd.read_pickle(pkl_path + "naics_occupation_part1.pickle")
df_occupation_2 = pd.read_pickle(pkl_path + "naics_occupation_part2.pickle")
df_occupation_3 = pd.read_pickle(pkl_path + "naics_occupation_part3.pickle")
df_county = pd.read_pickle(pkl_path + "county.pickle")

# Transforming the data

In [5]:
# def add_zeros(code):
#     if len(code) == 3:
#         return '00' + code
#     elif len(code) == 4:
#         return '0' + code
#     elif len(code) == 1:
#         return '0000' + code
#     return code

In [6]:
# Filling 0 to get same length for all FIPS
# df_pattern['FIPS'] = df_pattern['FIPS'].astype(str)
# unique_lengths = df_pattern['FIPS'].apply(len).unique()
# unique_lengths

# df_pattern['FIPS'] = df_pattern['FIPS'].apply(add_zeros)

In [7]:
# Merging Occupation Datasets
df_occupation = pd.concat([df_occupation_1, df_occupation_2, df_occupation_3], ignore_index=True)
df_occupation.head(5)

,FIPS,State_GEOID,naics,NAICS_TITLE,emp_total_county_naics,OCC_CODE,OCC_TITLE,emp_occupation,state_name
0,12999,12,5613,Employment Services,1436559,49-9071,"Maintenance and Repair Workers, General",20639.514235,NaN
1,06999,6,5613,Employment Services,729335,49-9071,"Maintenance and Repair Workers, General",9414.167765,NaN
2,36999,36,5613,Employment Services,308333,49-9071,"Maintenance and Repair Workers, General",8332.850279,NaN
3,06037,6,8111,Automotive Repair and Maintenance,25081,49-3023,Automotive Service Technicians and Mechanics,5913.423292,California
4,48999,48,5613,Employment Services,340926,49-9071,"Maintenance and Repair Workers, General",5770.378034,NaN


In [8]:
# Keeping only the necessary columns

# GDP
df_gdp_filtered = gdp_data.drop(gdp_data[['TableName', 'LineCode', 'Unit', 'Region', '2017', '2018', '2019', '2020', '2021', 'GeoName', 'IndustryClassification', 'Description']], axis=1)
df_gdp_filtered = df_gdp_filtered.rename(columns={'2022': 'current_gdp'})

df_gdp_filtered['FIPS'] = df_gdp_filtered['FIPS'].astype(str)
df_gdp_filtered = df_gdp_filtered[df_gdp_filtered['FIPS'].str.len() == 5] # Filtering for FIPS with 5 numbers --> need it on county level

# Naics Pattern
df_pattern_filtered = df_pattern.drop(columns=['emp_nf', 'qp1_nf', 'qp1', 'ap_nf',
       'n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
       'n500_999', 'n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4', 'naics_2', 'State_GEOID',	'County_GEOID'])
df_pattern_filtered = df_pattern_filtered.rename(columns={'DESCRIPTION': 'naics_describtion'})

df_pattern_filtered['FIPS'] = df_pattern_filtered['FIPS'].astype(str)
df_pattern_filtered = df_pattern_filtered[df_pattern_filtered['FIPS'].str.len() == 5] # Filtering for FIPS with 5 numbers

# Naics Occupation
df_occupation_filtered = df_occupation.drop(columns=['State_GEOID', 'NAICS_TITLE', 'emp_total_county_naics', 'state_name'])

df_occupation_filtered['FIPS'] = df_occupation_filtered['FIPS'].astype(str)
df_occupation_filtered = df_occupation_filtered[df_occupation_filtered['FIPS'].str.len() == 5] # Filtering for FIPS with 5 numbers

In [9]:
# # Define the relevant NAICS code prefixes as strings
# relevant_naics_prefixes = ['21', '23', '31', '32', '33']  # Sectors for Mining, Construction, Manufacturing

# # Convert relevant prefixes to a tuple for startswith
# relevant_naics_prefixes = tuple(relevant_naics_prefixes)

# # Filter df_gdp_filtered
# df_gdp_filtered = df_gdp_filtered[
#     df_gdp_filtered['naics'].astype(str).str.startswith(relevant_naics_prefixes)
# ]

# # Filter df_pattern_filtered
# df_pattern_filtered = df_pattern_filtered[
#     df_pattern_filtered['naics'].astype(str).str.startswith(relevant_naics_prefixes)
# ]

# # Filter df_occupation_filtered
# df_occupation_filtered = df_occupation_filtered[
#     df_occupation_filtered['naics'].astype(str).str.startswith(relevant_naics_prefixes)
# ]

In [10]:
# Merge the result with df_occupation_filtered on FIPS and naics
df_naics = df_pattern_filtered.merge(
    df_occupation_filtered,
    on=['FIPS', 'naics'],
    how='left',
    suffixes=('', '_occupation')
)

In [11]:
df_naics.head()

,FIPS,naics,naics_describtion,emp,ap,est,OCC_CODE,OCC_TITLE,emp_occupation
0,10001,2131,Support Activities for Mining,54,7136,5,49-9041,Industrial Machinery Mechanics,1.672731
1,10001,2131,Support Activities for Mining,54,7136,5,51-4121,"Welders, Cutters, Solderers, and Brazers",0.650311
2,10001,2131,Support Activities for Mining,54,7136,5,49-3042,"Mobile Heavy Equipment Mechanics, Except Engines",0.337553
3,10001,2131,Support Activities for Mining,54,7136,5,49-3031,Bus and Truck Mechanics and Diesel Engine Spec...,0.295242
4,10001,2131,Support Activities for Mining,54,7136,5,49-9071,"Maintenance and Repair Workers, General",0.271479


In [22]:
# Group by 'FIPS' and 'naics', then sum the specified columns
df_naics_agg = df_naics.groupby(['naics', 'naics_describtion']).agg({
    'ap': 'sum',
    'est': 'sum',
    'emp': 'sum'
}).reset_index()

df_naics_agg.head()

,naics,naics_describtion,ap,est,emp
0,1133,Logging,14144703,44479,273558
1,2111,Oil and Gas Extraction,160920430,65731,1086954
2,2121,Coal Mining,28820400,5325,338280
3,2122,Metal Ore Mining,47106670,2210,468182
4,2123,Nonmetallic Mineral Mining and Quarrying,88577161,69455,1183116


In [45]:
# Generate random values for each NAICS and assign them to each entry in the group
np.random.seed(1)

random_values = {naics: np.random.randint(0, 11) for naics in df_naics_agg['naics'].unique()}
df_naics_agg['tool_consumption'] = df_naics_agg['naics'].map(random_values)
df_naics_agg.sort_values(by='emp', ascending=False).head(10)

,naics,naics_describtion,ap,est,emp,tool_consumption
64,5613,Employment Services,17594424274,2575773,316290486,6
62,5413,"Architectural, Engineering, and Related Services",4056815914,3014994,43360446,9
13,2382,Building Equipment Contractors,2471259455,3208151,36373647,2
35,3330A1,"3331, 3332, 3334, 3339",1511893370,362952,20220626,3
24,3261,Plastics Product Manufacturing,1112538162,281892,19934743,6
43,3363,Motor Vehicle Parts Manufacturing,1107920522,134988,19280562,1
12,2381,"Foundation, Structure, and Building Exterior C...",1123358347,2190290,18616472,5
65,5617,Services to Buildings and Dwellings,641252104,1986470,17649729,9
67,8111,Automotive Repair and Maintenance,613043765,2579928,14193850,7
9,2371,Utility System Construction,988834457,381603,12313454,9


In [46]:
df_gdp_agg = df_gdp_filtered.groupby('FIPS').agg({
    'current_gdp': 'sum'
}).reset_index()
df_gdp_agg.head()

,FIPS,current_gdp
0,10000,355577213.0
1,10001,22652836.0
2,10003,243477316.0
3,10005,43727204.0
4,11000,574989171.0


# Ranking the most important industries

In [47]:
df_rank = df_naics_agg.copy()

for column in df_rank.columns[df_rank.columns.get_loc('ap'):]:
    rank_column_name = f'rank_{column}'
    df_rank[rank_column_name] = df_rank[column].rank(method='min', ascending=False).astype(int)

df_rank.sort_values(by='rank_emp').head()

,naics,naics_describtion,ap,est,emp,tool_consumption,rank_ap,rank_est,rank_emp,rank_tool_consumption
64,5613,Employment Services,17594424274,2575773,316290486,6,1,4,1,33
62,5413,"Architectural, Engineering, and Related Services",4056815914,3014994,43360446,9,2,2,2,5
13,2382,Building Equipment Contractors,2471259455,3208151,36373647,2,3,1,3,53
35,3330A1,"3331, 3332, 3334, 3339",1511893370,362952,20220626,3,4,14,4,48
24,3261,Plastics Product Manufacturing,1112538162,281892,19934743,6,6,17,5,33


In [48]:
weights = {
    'rank_tool_consumption': 0.5,
    'rank_emp': 0.2,
    'rank_ap': 0.1,
    'rank_est': 0.2
}

In [49]:
# calculate the weighted sum
df_rank['Weighted_Sum'] = (df_rank['rank_tool_consumption'] * weights['rank_tool_consumption'] +
                          df_rank['rank_emp'] * weights['rank_emp'] +
                          df_rank['rank_ap'] * weights['rank_ap'] +
                          df_rank['rank_est'] * weights['rank_est'])
df_rank = df_rank.sort_values(by='Weighted_Sum', ascending=True)

In [50]:
df_rank.head(10)

,naics,naics_describtion,ap,est,emp,tool_consumption,rank_ap,rank_est,rank_emp,rank_tool_consumption,Weighted_Sum
62,5413,"Architectural, Engineering, and Related Services",4056815914,3014994,43360446,9,2,2,2,5,3.5
65,5617,Services to Buildings and Dwellings,641252104,1986470,17649729,9,16,6,8,5,6.9
9,2371,Utility System Construction,988834457,381603,12313454,9,8,13,10,5,7.9
49,3399,Other Miscellaneous Manufacturing,453204190,472189,7837580,10,21,11,19,1,8.6
33,3327,"Machine Shops; Turned Product; and Screw, Nut,...",611343199,636266,10365797,8,18,9,14,17,14.9
44,3364,Aerospace Product and Parts Manufacturing,946714787,35629,9811934,9,9,43,16,5,15.2
67,8111,Automotive Repair and Maintenance,613043765,2579928,14193850,7,17,3,9,23,15.6
37,3335,Metalworking Machinery Manufacturing,235282422,150709,3546341,9,31,23,30,5,16.2
47,3370A1,"3371, 3372",385685233,331463,7914077,8,24,15,18,17,17.5
64,5613,Employment Services,17594424274,2575773,316290486,6,1,4,1,33,17.6


# Data Preprocessing

In [ ]:
df_final = df_final.copy()

In [ ]:
# Impute Missing GDP with 0
df_final['current_gdp'].fillna(0, inplace=True)

In [ ]:
# Choosing relevant features
features = ['ap', 'est', 'emp_occupation', 'current_gdp']
X = df_final[features]

In [ ]:
# Scaling the data to make them comparable
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA

In [ ]:
pca = PCA(n_components=2) # the elbow method showed 2 PC are best
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, marker='o')
plt.title('Explained Variance per Priciple Component')
plt.xlabel('Principle Component')
plt.ylabel('Explained Variance')
plt.show()

# Clustering

In [ ]:
inertia = []
cluster_range = range(1, 11)

for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_pca)
    inertia.append(kmeans.inertia_)

# Elbow-Method to visualize
plt.figure(figsize=(8, 6))
plt.plot(cluster_range, inertia, marker='o')
plt.title('Elbow-Method to determine the best amount of clusters')
plt.xlabel('Clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_pca)
df_final['cluster'] = clusters

plt.figure(figsize=(8, 6))

for cluster_label in sorted(set(clusters)):
    plt.scatter(
        X_pca[clusters == cluster_label, 0],
        X_pca[clusters == cluster_label, 1],
        label=f'Cluster {cluster_label}',
        alpha=0.7,
        edgecolors='k'
    )

plt.title('K-Means Clustering with 3 Clusters')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

# Interpretation

In [ ]:
df_final.head()

In [ ]:
cluster_summary = df_final.groupby('cluster')[['ap', 'est', 'emp_occupation', 'current_gdp']].mean()
print(cluster_summary)

In [ ]:
best_cluster = cluster_summary['emp_occupation'].idxmax()  # Cluster with highest emp_occ
best_counties = df_final[df_final['cluster'] == best_cluster]
print(best_counties[['FIPS', 'ap', 'est', 'emp_occupation', 'current_gdp', 'cluster']])

In [ ]:
df_result = df_final[df_final['cluster']==1]
df_result.head()

In [ ]:
df_result.info()